In [ ]:
import numpy as np
import librosa
import soundfile as sf
from sklearn.decomposition import FastICA
from scipy.signal import stft, istft

def separate_ica_single_channel(audio_path, n_components=2):
    # 1. Charger l'audio
    y, sr = librosa.load(audio_path, sr=None)
    
    # 2. Calculer la STFT (on travaille sur la magnitude)
    f, t, Zxx = stft(y, fs=sr, nperseg=1024)
    magnitude = np.abs(Zxx)
    phase = np.angle(Zxx)
    
    # 3. Appliquer l'ICA
    # On considère chaque frame temporelle comme une observation
    # et chaque bin de fréquence comme une variable
    ica = FastICA(n_components=n_components, random_state=42, whiten='unit-variance')
    
    # On transpose pour avoir (temps, fréquences)
    S_tilde = ica.fit_transform(magnitude.T)  # Sources estimées (activations temporelles)
    A_tilde = ica.mixing_ # Signatures spectrales
    
    # 4. Reconstruction des deux sources
    # Source 1
    mag_s1 = np.outer(A_tilde[:, 0], S_tilde[:, 0]).T
    # Source 2
    mag_s2 = np.outer(A_tilde[:, 1], S_tilde[:, 1]).T
    
    # 5. Retour au signal temporel (on utilise la phase du mélange original)
    _, res_s1 = istft(mag_s1 * np.exp(1j * phase), fs=sr)
    _, res_s2 = istft(mag_s2 * np.exp(1j * phase), fs=sr)
    
    return res_s1, res_s2, sr

# Test sur un fichier
path_mix = "data/train/0001/mix_snr_0.wav"
s1, s2, sr = separate_ica_single_channel(path_mix)

In [ ]:
def calculate_si_sdr(reference, estimation):
    """Calcul du Scale-Invariant Signal-to-Distortion Ratio"""
    reference = reference[:len(estimation)]
    estimation = estimation[:len(reference)]
    
    dot_product = np.dot(reference, estimation)
    norm_ref = np.linalg.norm(reference)**2
    
    projection = (dot_product / norm_ref) * reference
    noise = estimation - projection
    
    si_sdr = 10 * np.log10(np.linalg.norm(projection)**2 / np.linalg.norm(noise)**2)
    return si_sdr

# Chargement des vérités terrain pour évaluation
v_true, _ = librosa.load("data/train/0001/voice.wav", sr=sr)
n_true, _ = librosa.load("data/train/0001/noise.wav", sr=sr)

# L'ICA ne sait pas laquelle est la voix et lequel est le bruit (ambiguïté de permutation)
score_1 = calculate_si_sdr(v_true, s1)
score_2 = calculate_si_sdr(v_true, s2)

print(f"Meilleur SI-SDR trouvé par ICA : {max(score_1, score_2):.2f} dB")